In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install addict
!pip install earthpy

In [ ]:
from addict import Dict
from pathlib import Path

data_dir = Path("/content/drive/MyDrive/Colab Notebooks/Tesi/deforestation_mapping-tesi/datadrive/forest")
process_dir = data_dir / "processed"

args = Dict({
    "batch_size": 8,
    "run_name": "demo",
    "epochs": 100,
    "save_every": 50,
    "loss_type": "dice",
    "device": "cuda:0"
})

In [ ]:
%cd drive/MyDrive/Colab Notebooks/Tesi/deforestation_mapping-tesi

In [ ]:
from matplotlib.image import imread
from PIL import Image
import matplotlib.pyplot as plt
import ee
import earthpy.plot as ep
import numpy as np
import cv2

In [ ]:
!pip install -U imagecodecs

In [ ]:
import tifffile
img = tifffile.imread('/content/drive/MyDrive/Colab Notebooks/Tesi/deforestation_mapping-tesi/datadrive/forest/processed/Test/images/S2A_MSIL2A_20200217T141651_N0209_R010_T20LQN_20200217T160045_16_14.tif')
plt.figure(figsize=(12,12))
# changing the index in the square brackets you can visualize the 4 bands
plt.imshow(img[...,2])

In [ ]:
# Per vedere immagine RGB del dataset sulle foreste
image = np.dstack((img[...,0],img[...,1],img[...,2]))

image2 = image.transpose(2,0,1)

ep.plot_rgb(image2,
            title="RGB Composite Image",
            stretch=True,
            str_clip=1)
plt.show()
mask = tifffile.imread('/content/drive/MyDrive/Colab Notebooks/Tesi/deforestation_mapping-tesi/datadrive/forest/processed/Test/masks/S2A_MSIL2A_20200217T141651_N0209_R010_T20LQN_20200217T160045_16_14.tif')
plt.figure(figsize=(10,10))
plt.imshow(mask)

In [ ]:
x_img = img.reshape(-1,4)
print('The shape after reshaping of the image is ', x_img.shape)

image = np.dstack((img[...,0],img[...,1],img[...,2]))
print('The shape of the image (only rgb) is ', image.shape)
x_image = image.reshape(-1,3)

In [ ]:
# Calcolo ndvi
red = img[...,0]
nir = img[...,3]
ndvi = (nir.astype(float) - red.astype(float)) / (nir + red)

# Aggiungo la banda all'immagine originale
new = np.dstack((img, ndvi))
new.shape
plt.figure(figsize=(12,12))
# changing the index in the square brackets you can visualize the 5 bands
plt.imshow(new[...,4])

new_img = new.reshape(-1,5)
print('The shape after reshaping of the image is ', new_img.shape)

ndvi_res = ndvi.reshape(-1,1)

In [ ]:
ep.plot_bands(ndvi, cmap="RdYlGn", cols=1, title='NDVI', vmin=0, vmax=1)

# Create classes and apply to NDVI results
ndvi_class_bins = [-np.inf, 0.5, 0.6, 0.7, 0.82, np.inf]
ndvi_landsat_class = np.digitize(ndvi, ndvi_class_bins)

# Apply the nodata mask to the newly classified NDVI data
ndvi_landsat_class = np.ma.masked_where(
    np.ma.getmask(ndvi), ndvi_landsat_class
)
np.unique(ndvi_landsat_class)

In [ ]:
# Calcolo ndwi
green = img[...,1]
nir = img[...,3]
ndwi = (nir.astype(float) - green.astype(float)) / (nir + green)

# Aggiungo la banda all'immagine originale
new_band = np.dstack((img, ndwi))
new_band.shape
plt.figure(figsize=(12,12))
# changing the index in the square brackets you can visualize the 5 bands
plt.imshow(new_band[...,4])

new_img = new_band.reshape(-1,5)
print('The shape after reshaping of the image is ', new_img.shape)

ndwi_res = ndwi.reshape(-1,1)

In [ ]:
ep.plot_bands(ndwi, cmap="RdYlGn", cols=1, title='NDWI', vmin=0, vmax=1)

# Create classes and apply to NDWI results
ndwi_class_bins = [-np.inf, 0.5, 0.6, 0.7, 0.82, np.inf]
ndwi_landsat_class = np.digitize(ndwi, ndwi_class_bins)

# Apply the nodata mask to the newly classified NDWI data
ndwi_landsat_class = np.ma.masked_where(
    np.ma.getmask(ndvi), ndwi_landsat_class
)
np.unique(ndwi_landsat_class)

In [ ]:
# Calcolo msavi
red = img[...,0]
nir = img[...,3]
msavi = nir.astype(float) + 0.5 - (0.5 * np.sqrt((2 * nir.astype(float) + 1)**2 - 8 * (nir.astype(float) - (2 * red.astype(float)))))

# Aggiungo la banda all'immagine originale
new_band = np.dstack((img, msavi))
new_band.shape
plt.figure(figsize=(12,12))
# changing the index in the square brackets you can visualize the 5 bands
plt.imshow(new_band[...,4])

new_img = new_band.reshape(-1,4)
print('The shape after reshaping of the image is ', new_img.shape)

ndwi_res = ndwi.reshape(-1,1)


In [ ]:
ep.plot_bands(msavi, cmap="RdYlGn", cols=1, title='MSAVI', vmin=-1, vmax=1)

# Create classes and apply to MSAVI results
msavi_class_bins = [-np.inf, 0.5, 0.6, 0.7, 0.82, np.inf]
msavi_landsat_class = np.digitize(msavi, msavi_class_bins)

# Apply the nodata mask to the newly classified NDWI data
msavi_landsat_class = np.ma.masked_where(
    np.ma.getmask(msavi), msavi_landsat_class
)
np.unique(msavi_landsat_class)

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
# Define color map
nbr_colors = ["gray", "y", "yellowgreen", "g", "darkgreen"]
nbr_cmap = ListedColormap(nbr_colors)

# Define class names
ndvi_cat_names = [
    "No Vegetation",
    "Bare Area",
    "Low Vegetation",
    "Moderate Vegetation",
    "High Vegetation",
]

# Get list of classes
classes = np.unique(ndvi_landsat_class)
classes = classes.tolist()
# The mask returns a value of none in the classes. remove that
classes = classes[0:5]

# Plot your data
fig, ax = plt.subplots(figsize=(12, 12))
im = ax.imshow(ndvi_landsat_class, cmap=nbr_cmap)

ep.draw_legend(im_ax=im, classes=classes, titles=ndvi_cat_names)
ax.set_title(
    "Normalized Difference Vegetation Index (NDVI) Classes",
    fontsize=14,
)
ax.set_axis_off()

# Auto adjust subplot to fit figure size
plt.tight_layout()

In [ ]:
from sklearn.cluster import KMeans, DBSCAN

In [ ]:
# Delle seguenti celle fino a kmeans lanciarne una alla volta, se sono ad una sola banda togliere il -1 dal reshape della predict

In [ ]:
image_rgbndvi = np.dstack((image, ndvi))
print('The shape of the image (rgb and ndvi) is ', image_rgbndvi.shape)
xx_image = image_rgbndvi.reshape(-1,4)

In [ ]:
image_rgbbands = np.dstack((image_rgbndvi, ndwi))
print('The shape of the image (rgb and ndvi) is ', image_rgbbands.shape)
xx_image = image_rgbbands.reshape(-1,5)

In [ ]:
print('The shape of the image (msavi) is ', msavi.shape)
msavi_image = msavi.reshape(-1,1)
print(msavi_image.shape)

In [ ]:
image_rgbmsavi = np.dstack((image, msavi))
print('The shape of the image (rgb and msavi) is ', image_rgbmsavi.shape)
xx_image = image_rgbmsavi.reshape(-1,4)

In [ ]:
image_total = np.dstack((image,ndvi,ndwi, msavi))
print('The shape of the image (rgb and msavi) is ', image_total.shape)
xx_image = image_total.reshape(-1,6)

In [ ]:
image_vi = np.dstack((ndvi,ndwi, msavi))
print('The shape of the image (rgb and msavi) is ', image_vi.shape)
xx_image = image_vi.reshape(-1,3)

In [ ]:
km = KMeans(7)
km.fit(xx_image)

# Usando tutte e 4 le bande a disposizione dall'immagine si ottengono queste situazioni:
# k=2 distingue solo nuvole e non nuvole
# k=3 distingue nuvole, ombre e background
# k=4 separa bene foresta da non foresta (es. coltivazioni), ma rimane il problema di nuvole e ombre (migliore delle 3)

# Usando solo le bande RGB si ottengono queste situazioni:
# k=2 distingue solo nuvole e non nuvole
# k=3 distingue nuvole piene, unisce nuovole rade e coltivazioni e unisce ombre e foresta (miglore delle 3)
# k=4 come k=3 ma delinea meglio le nuvole piene

# Usando tutte e 4 le bande più l'NDVI si ottengono queste situazioni:
# Come il caso 1

# Usando solo NDVI si ottengono queste situazioni:
# k=2 distingue foresta da non foresta con errori sulle coltivazioni, nella non foresta include le nuvole
# k=3 distingue foresta, nuvole piene e accorpa coltivzioni e nuvole chiare
# k=4 separa bene foresta da non foresta (es. coltivazioni), ma rimane il problema di nuvole (migliore delle 3) (coltivazioni con nuvole chiare)

# Usando RGB e NDVI
# k=2 distingue solo nuvole e non nuvole
# k=3 distingue foresta, nuvole piene e accorpa coltivzioni e nuvole chiare
# k=4 separa bene foresta da non foresta (es. coltivazioni), ma rimane il problema di nuvole (migliore delle 3) (coltivazioni con nuvole chiare)

# Usando RGB e NDVI e NDWI (per water bodies, quindi non so se utile)
# k=2 distingue solo nuvole e non nuvole
# k=3 distingue foresta, nuvole piene e accorpa coltivzioni e nuvole chiare
# k=4 non cambia rispetto a k=3 (coltivazioni con nuvole chiare)

# Usando solo Msavi
# k=2 distingue solo nuvole e non nuvole e accorpa alcuni elementi di campo alle nuvole
# k=3 distingue foresta, nuvole piene e accorpa coltivzioni e nuvole chiare
# k=4 distigue foresta da non foresta ma confonde alcune zone di campo con le nuvole chiare e viceversa

# Usando RGB e MSAVI
# k=2 distingue solo nuvole e non nuvole
# k=3 distingue foresta, nuvole piene e accorpa coltivzioni e nuvole chiare
# k=4 distigue foresta da non foresta ma confonde alcune zone di campo con le nuvole chiare e viceversa (migliore)

# Usando RGB, NDVI, NDWI e MSAVI
# k=2 distingue solo nuvole e non nuvole
# k=3 distingue foresta, nuvole piene e accorpa coltivzioni e nuvole chiare
# k=4 distigue foresta da non foresta ma confonde alcune zone di campo con le nuvole chiare e viceversa (migliore)

# Usando NDVI, NDWI e MSAVI
# k=2 distingue foresta da non foresta ma accorpa alcuni campi nella foresta
# k=3 distingue foresta, nuvole piene e accorpa coltivzioni e nuvole chiare
# k=4 distigue foresta da non foresta ma confonde alcune zone di campo con le nuvole chiare e viceversa (migliore)

In [ ]:
seg = km.predict(xx_image).reshape(image_vi.shape[:-1])
seg

In [ ]:
plt.imshow(seg)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(xx_image)
scaled_features

In [ ]:
kmeans_kwargs = {
     "init": "random",
     "n_init": 10,
     "max_iter": 300,
     "random_state": 42,
}
# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()